In [5]:
import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

data_dir = os.path.join('..','..','data')

with open(os.path.join(data_dir,'maps','Nakhon-Si-Thammarat-choropleth-total.geojson')) as f:
    shapefile = json.load(f)

In [6]:
shapefile['features'][0]['properties']

{'AP_EN': 'Phrom Khiri',
 'AP_IDN': '8002',
 'AP_TN': 'พรหมคีรี',
 'Area': 1070,
 'Avg_BI': 20,
 'Breeding_site_counts': 678,
 'Breteau_index': {'2011': 0,
  '2012': 0,
  '2013': 0,
  '2014': 50,
  '2015': 33,
  '2016': 0,
  '2017': 0},
 'Dengue_cases': 25,
 'Density': 6,
 'Density_population': 8,
 'PV_EN': 'Nakhon Si Thammarat',
 'PV_IDN': '80',
 'PV_TN': 'นครศรีธรรมราช',
 'Population': 8387,
 'TB_EN': 'Thon Hong',
 'TB_IDN': '800204',
 'TB_TN': 'ทอนหงส์'}

### Detected Breeding Sites

In [8]:
df_detect = pd.read_csv(os.path.join(data_dir,'breeding-sites','csv','addrcode-index','detection.csv')) 
df_detect['date'] = pd.to_datetime(df_detect['date'], format='%Y-%m')
df_detect['addrcode'] = df_detect['addrcode'].astype('int')
df_detect = df_detect.set_index('date')
df_detect = df_detect.sort_index()
df_detect.head()

,addrcode,bin,bowl,bucket,cup,jar,pottedplant,tire,vase,total
date,,,,,,,,,,
2016-02-01,810602,270,1,0,35,38,24,2,139,509
2016-02-01,810503,307,10,1,9,32,38,1,104,502
2016-02-01,810507,253,3,0,16,80,55,2,146,555
2016-02-01,810205,263,2,0,21,33,28,1,112,460
2016-02-01,810504,394,4,1,26,76,105,3,190,799


In [43]:
x = []
cnt = 0
for feature in shapefile['features']:
    
    prop = feature['properties']
    
    addrcode = prop['TB_IDN']
    
    brd_site_classes = df_detect.loc[df_detect['addrcode'] == int(addrcode)][['bin','bowl','bucket','cup','jar','pottedplant','tire','vase']]
    
    if len(brd_site_classes) == 0:
        feature['properties']['Breeding_site_classes'] = {
            'bin': 0,
            'bowl': 0,
            'bucket': 0,
            'cup':0,
            'jar': 0,
            'pottedplant': 0,
            'tire': 0,
            'vase':0
        }
    else:
        feature['properties']['Breeding_site_classes'] = {
            'bin': int(brd_site_classes.bin.values[0]),
            'bowl': int(brd_site_classes.bowl.values[0]),
            'bucket': int(brd_site_classes.bucket.values[0]),
            'cup': int(brd_site_classes.cup.values[0]),
            'jar': int(brd_site_classes.jar.values[0]),
            'pottedplant': int(brd_site_classes.pottedplant.values[0]),
            'tire': int(brd_site_classes.tire.values[0]),
            'vase': int(brd_site_classes.vase.values[0]),
        }

In [44]:
with open(os.path.join(data_dir, 'maps', 'Nakhon-Si-Thammarat-choropleth-each-class.geojson'), 'w') as FILE:
    json.dump(shapefile, FILE, indent=4, ensure_ascii=False, sort_keys=True)